# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
!python -m pip install scikit-learn --upgrade
# !python -m pip install nltk

In [2]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
import re
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\josep\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\josep\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\josep\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', con=engine)
X = df.loc[:, 'message']
y = df.iloc[:, 4:]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    # Split text into words using NLTK
    words = word_tokenize(text)
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    # Reduce words to their root form
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    # Lemmatize verbs by specifying pos
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed]
    # Reduce words to their stems
    stemmed = [PorterStemmer().stem(w) for w in lemmed]
    tokenized = stemmed
    return tokenized

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
random_forest_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
random_forest_pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x00000229DA38BEE0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
y_pred = random_forest_pipeline.predict(X_test)

In [9]:
print(classification_report(y_test, y_pred, zero_division=0, target_names=y.columns))

                        precision    recall  f1-score   support

               related       0.84      0.95      0.89      5007
               request       0.83      0.52      0.64      1095
                 offer       0.00      0.00      0.00        39
           aid_related       0.77      0.69      0.73      2723
          medical_help       0.62      0.09      0.16       552
      medical_products       0.73      0.10      0.18       331
     search_and_rescue       0.71      0.06      0.11       169
              security       0.00      0.00      0.00       129
              military       0.78      0.03      0.06       220
           child_alone       0.00      0.00      0.00         0
                 water       0.86      0.38      0.52       444
                  food       0.82      0.67      0.74       720
               shelter       0.83      0.38      0.52       590
              clothing       0.80      0.15      0.26       105
                 money       0.80      

### 6. Improve your model
Use grid search to find better parameters. 

In [13]:
parameters = {
#     'tfidf__use_idf': (True, False),
#     'clf__estimator__min_samples_split': [2, 3]
    'clf__estimator__n_estimators': [5]
}

rf_cv_gs = GridSearchCV(random_forest_pipeline, param_grid=parameters, scoring = ['accuracy', 'f1_macro'], verbose=3, cv=2, 
                                return_train_score=True, refit=False)

In [14]:
rf_cv_gs.fit(X_train, y_train)

Fitting 2 folds for each of 1 candidates, totalling 2 fits


C:\Users\josep\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
C:\Users\josep\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


[CV 1/2] END clf__estimator__n_estimators=5; accuracy: (train=0.753, test=0.200) f1_macro: (train=0.884, test=0.262) total time= 8.1min


C:\Users\josep\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


[CV 2/2] END clf__estimator__n_estimators=5; accuracy: (train=0.746, test=0.191) f1_macro: (train=0.878, test=0.258) total time= 8.8min


C:\Users\josep\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\metrics\_classification.py:1570: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


GridSearchCV(cv=2,
             estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x00000229DA38BEE0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [5]}, refit=False,
             return_train_score=True, scoring=['accuracy', 'f1_macro'],
             verbose=3)

In [15]:
rf_cv_gs.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
best_estimator = rf_cv_gs.best_estimator_

In [ ]:
y_pred = best_estimator.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, zero_division=0, target_names=y.columns))

In [ ]:
# printing the evaluation scores
cvres = random_forest_cv.cv_results_
for mean_f1_macro, mean_accuracy, params in zip(cvres['mean_test_f1_macro'], cvres['mean_test_accuracy'], cvres['params']):
    print(mean_f1_macro, mean_accuracy, params)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
decision_tree_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

In [ ]:
decision_tree_pipeline.fit(X_train, y_train)

In [ ]:
y_pred = decision_tree_pipeline.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, zero_division=0, target_names=y.columns))

In [ ]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [ ]:
class TextLengthExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        text_len = len(text.strip())
        return text_len

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [ ]:
# With Column Transformer
X = df.loc[:, ['message', 'genre']]
# message = ['message'] Why thi didn't work?
message = 'message'
genre = ['genre']

full_pipeline = Pipeline([
    ('features', FeatureUnion([
        ('col_xfmr', ColumnTransformer([
            ('vect', CountVectorizer(tokenizer=tokenize), message),
            ('tfidf', TfidfTransformer(), message),
            ('cat', OneHotEncoder(), genre),
        ])),

        ('txt_length', TextLengthExtractor()),
        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

In [ ]:
decision_tree_pipeline.fit(X_train, y_train)

In [ ]:
y_pred = decision_tree_pipeline.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, zero_division=0, target_names=y.columns))

In [ ]:
parameters = {
    'tfidf__use_idf': (True, False),
    'clf__estimator__min_samples_split': [2, 3]
}

full_pipeline_cv = GridSearchCV(pipeline, param_grid=parameters, scoring = ['accuracy', 'f1_macro'], refit=False, verbose=True)

In [ ]:
full_pipeline_cv.fit(X_train, y_train)

In [ ]:
full_pipeline_cv.best_params_

In [ ]:
best_estimator = full_pipeline_cv.best_estimator_
y_pred = best_estimator.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred, zero_division=0, target_names=y.columns))

In [ ]:
# printing the evaluation scores
cvres = full_pipeline_cv.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

### 9. Export your model as a pickle file

In [ ]:
import joblib
joblib.dump(random_forest_pipeline, "forest_reg.pkl")
joblib.dump(decision_tree_pipeline, "lin_reg.pkl")
joblib.dump(best_estimator, "tree_reg.pkl")

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.